In [ ]:
#!/usr/bin/env python3
"""
Performance benchmark for Fast Depth Pipeline
Tests different configurations to find optimal settings
"""

import time
import numpy as np
import torch
from pathlib import Path
import sys
sys.path.append('.')
from fast_depth_pipeline import FastDepthPipeline

def create_test_image(size=(512, 512)):
    """Create a test image for benchmarking"""
    # Create a simple test pattern
    img = np.random.randint(0, 255, (*size, 3), dtype=np.uint8)
    # Add some structure (circles, lines)
    center = (size[0]//2, size[1]//2)
    y, x = np.ogrid[:size[0], :size[1]]
    mask = (x - center[0])**2 + (y - center[1])**2 < (size[0]//4)**2
    img[mask] = [255, 0, 0]  # Red circle
    
    return img

def benchmark_configuration(config_name, **kwargs):
    """Benchmark a specific configuration"""
    print(f"\n🧪 Testing {config_name}...")
    
    try:
        # Initialize pipeline
        pipeline = FastDepthPipeline(**kwargs)
        
        # Create test image
        test_img = create_test_image()
        test_path = Path("test_image.jpg")
        
        # Save test image
        from PIL import Image
        Image.fromarray(test_img).save(test_path)
        
        # Warmup run
        pipeline.process_image(test_path, skip_visual=True)
        
        # Benchmark runs
        times = []
        for i in range(5):
            start = time.time()
            result = pipeline.process_image(test_path, skip_visual=True)
            end = time.time()
            times.append(end - start)
        
        # Calculate statistics
        avg_time = np.mean(times)
        std_time = np.std(times)
        
        print(f"   ⏱️  Average time: {avg_time:.3f}s (±{std_time:.3f}s)")
        print(f"   📊 Points generated: {result['meta']['num_points']}")
        print(f"   🎯 Shape detected: {result['analysis']['type']}")
        
        # Cleanup
        test_path.unlink()
        
        return avg_time, result
        
    except Exception as e:
        print(f"   ❌ Error: {e}")
        return None, None

def main():
    """Run comprehensive benchmarks"""
    print("🏃‍♂️ Fast Depth Pipeline Benchmark")
    print("=" * 50)
    
    # Check Metal availability
    metal_available = torch.backends.mps.is_available()
    print(f"🔋 Metal backend available: {metal_available}")
    
    # Test configurations
    configs = [
        ("Ultra Fast (256px, sparse)", {
            "model_type": "midas_small",
            "input_size": 256,
            "use_metal": metal_available
        }),
        ("Fast (384px, sparse)", {
            "model_type": "midas_small", 
            "input_size": 384,
            "use_metal": metal_available
        }),
        ("Balanced (512px, sparse)", {
            "model_type": "midas_small",
            "input_size": 512,
            "use_metal": metal_available
        }),
        ("CPU Only (256px)", {
            "model_type": "midas_small",
            "input_size": 256,
            "use_metal": False
        })
    ]
    
    results = {}
    
    for config_name, config_params in configs:
        avg_time, result = benchmark_configuration(config_name, **config_params)
        if avg_time is not None:
            results[config_name] = avg_time
    
    # Summary
    print("\n📊 BENCHMARK RESULTS")
    print("=" * 50)
    
    if results:
        fastest_config = min(results, key=results.get)
        
        for config_name, avg_time in sorted(results.items(), key=lambda x: x[1]):
            speedup = results[fastest_config] / avg_time if avg_time > 0 else 1
            print(f"   {config_name}: {avg_time:.3f}s ({speedup:.1f}x)")
        
        print(f"\n🏆 Fastest configuration: {fastest_config}")
        print(f"🎯 Recommended for M2 Mac: Ultra Fast (256px, sparse)")
        
        # Performance tips
        print("\n💡 Performance Tips:")
        print("   • Use --size 256 for fastest processing")
        print("   • Use --sample-rate 8 for sparse point clouds")
        print("   • Use --skip-visual to avoid 3D rendering")
        print("   • Process images in batches for better efficiency")
        
        if metal_available:
            print("   • Metal backend is enabled for GPU acceleration")
        else:
            print("   • Consider enabling Metal backend for better performance")
    
    else:
        print("❌ No successful benchmark runs")

if __name__ == "__main__":
    main()